In [209]:
from bokeh.io import show,push_notebook, output_notebook
from bokeh.plotting import figure
from bokeh.palettes import viridis,Viridis256
from bokeh.layouts import column,widgetbox
from bokeh.models import (ColumnDataSource, HoverTool, LinearInterpolator,
                          CategoricalColorMapper,Slider,CustomJS,ColorBar,LinearColorMapper)
import pandas as pd
import calendar
import ipywidgets
from ipywidgets import interact
output_notebook()

Loading BokehJS ...

In [126]:
path="C:\\Program Files (x86)\Python36-32"
data=pd.read_csv(path+"\data.csv",
                 usecols=["tripduration","hour","month"])

In [213]:
data_temp=data.groupby("hour").size().reset_index(name="times")
data_temp["durations"]=data.groupby("hour").tripduration.mean().reset_index(name="durations").durations
data_int=data.groupby(["hour","month"]).size().reset_index(name="times")
data_int["durations"]=data.groupby(["hour","month"]).tripduration.mean().reset_index(name="durations").durations

PLOT_OPTS=dict(height=500,width=800)

source=ColumnDataSource(data=dict(
    x=data_int.times[data_int.month==1].tolist(),
    y=data_int.durations[data_int.month==1].tolist(),
    hours=[str(e) for e in list(data_int.hour.unique())]
))

def update(Month):
    new_data=dict(
        x=data_int.times[data_int.month==Month].tolist(),
        y=data_int.durations[data_int.month==Month].tolist(),
        hours=[str(e) for e in list(data_int.hour.unique())]
    )
    source.data=new_data
    p.title.text=calendar.month_name[Month]+" Bike Traffic by Hours"
    size_mapper=LinearInterpolator(
    x=[data_int.durations[data_int.month==Month].min(),
       data_int.durations[data_int.month==Month].max()],
    y=[5,50]
    )
    push_notebook()

size_mapper=LinearInterpolator(
    x=[data_int.durations[data_int.month==1].min(),
       data_int.durations[data_int.month==1].max()],
    y=[5,50]
)

color_mapper=CategoricalColorMapper(
    factors=[str(e) for e in list(data_int.hour.unique())],
    palette=viridis(len(list(data_int.hour.unique())))
)
color_mapper_lin=LinearColorMapper(
    palette=Viridis256,
    low=data_int.hour.min(),
    high=data_int.hour.max()
)


In [214]:
hover=HoverTool(tooltips="@hours")

p=figure(title="January Bike Traffic by Hours",tools=[hover],**PLOT_OPTS)
p.xaxis[0].axis_label = "Number of Trips"
p.yaxis[0].axis_label = "Duration"
p.circle(x="x", y="y",
         size={"field":"y","transform":size_mapper},
         color={"field":"hours","transform":color_mapper},
         alpha=0.5,source=source
)
p.add_layout(ColorBar(color_mapper=color_mapper_lin,location=(0,0),orientation="horizontal"),"below")

In [216]:
interact(update,Month=ipywidgets.IntSlider(min=1,max=12,step=1,value=1))
show(p,notebook_handle=True)

interactive(children=(IntSlider(value=1, description='Month', max=12, min=1), Output()), _dom_classes=('widget…